In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("OVER_train_augmented.csv")
df.head()

# GoogLeNet

In [ ]:
# Import necessary modules
from keras.optimizers import Adam
from keras.losses import CategoricalCrossentropy
from keras.metrics import CategoricalAccuracy
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Input, Dense, Dropout, Flatten, concatenate, AveragePooling2D


In [ ]:
def inception_module(x, filters_1x1, filters_3x3_reduce, filters_3x3, filters_5x5_reduce, filters_5x5, filters_pool_proj):
    path1 = Conv2D(filters_1x1, (1, 1), padding='same', activation='relu')(x)
    path2 = Conv2D(filters_3x3_reduce, (1, 1), padding='same', activation='relu')(x)
    path2 = Conv2D(filters_3x3, (3, 3), padding='same', activation='relu')(path2)
    path3 = Conv2D(filters_5x5_reduce, (1, 1), padding='same', activation='relu')(x)
    path3 = Conv2D(filters_5x5, (5, 5), padding='same', activation='relu')(path3)
    path4 = MaxPooling2D((3, 3), strides=(1, 1), padding='same')(x)
    path4 = Conv2D(filters_pool_proj, (1, 1), padding='same', activation='relu')(path4)
    return concatenate([path1, path2, path3, path4], axis=-1)

def create_google_model():
    input_layer = Input(shape=(227, 227, 3))

    x = Conv2D(64, (7, 7), strides=(2, 2), padding='same', activation='relu')(input_layer)
    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same')(x)
    x = Conv2D(64, (1, 1), strides=(1, 1), padding='same', activation='relu')(x)
    x = Conv2D(192, (3, 3), strides=(1, 1), padding='same', activation='relu')(x)
    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same')(x)
    x = inception_module(x, 64, 96, 128, 16, 32, 32)
    x = inception_module(x, 128, 128, 192, 32, 96, 64)
    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same')(x)
    x = inception_module(x, 192, 96, 208, 16, 48, 64)
    x = inception_module(x, 160, 112, 224, 24, 64, 64)
    x = inception_module(x, 128, 128, 256, 24, 64, 64)
    x = inception_module(x, 112, 144, 288, 32, 64, 64)
    x = inception_module(x, 256, 160, 320, 32, 128, 128)
    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same')(x)
    x = inception_module(x, 256, 160, 320, 32, 128, 128)
    x = inception_module(x, 384, 192, 384, 48, 128, 128)

    x = AveragePooling2D(pool_size=(7, 7), strides=(1, 1), padding='valid')(x)
    x = Dropout(0.4)(x)
    x = Flatten()(x)
    x = Dense(1000, activation='relu')(x)
    output_layer = Dense(8, activation='softmax')(x)

    # Define the model
    model_googlenet = Model(input_layer, output_layer)

    # Return the model
    return model_googlenet

ChatGPT for application 

In [ ]:
# Create your model
model_google = create_model()

# Compile the model
optimizer = Adam(learning_rate=0.001)
loss = CategoricalCrossentropy()
metrics = [CategoricalAccuracy()]

model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

# Fit the model to the training data
log = model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val))

# Evaluate the model on the test data
model.evaluate(X_test, y_test)

# Use the model to make predictions
predictions = model.predict(X_new)

# If your task is multi-class classification, you can get the class with the highest probability
import numpy as np
class_predictions = np.argmax(predictions, axis=-1)

NameError: name 'create_model' is not defined

random search + training the model

In [ ]:
# sklearn random serach function for a neural network
param_grid = {"optimizer": ["adam", "sgd", "rmsprop"],
                "learning_rate": [0.001, 0.01, 0.1],
                "momentum": [0.0, 0.2, 0.4, 0.6, 0.8, 0.9],
                "epochs": [10, 20, 30, 40, 50],
                "batch_size": [10, 20, 30, 40, 50],
                "activation": ["relu", "sigmoid", "tanh"],
                "dropout_rate": [0.0, 0.1, 0.2, 0.3, 0.4, 0.5],
                "weight_constraint": [1, 2, 3, 4, 5],
                
                "neurons": [1, 5, 10, 15, 20, 25, 30]}

def random_search(X_train, y_train, X_test, y_test, param_grid, n_iter_search=10, n_jobs=-1, cv=3, verbose=1, random_state=1): 
    # create a random search object
    random_search = RandomizedSearchCV(estimator=KerasClassifier(build_fn=create_google_model, verbose=0), 
                                        param_distributions=param_grid,
                                        n_iter=n_iter_search,
                                        n_jobs=n_jobs,
                                        cv=cv,
                                        verbose=verbose,
                                        random_state=random_state)
    # fit the random search model
    random_search.fit(X_train, y_train)
    # print results
    print("Best Score: ", random_search.best_score_)
    print("Best Params: ", random_search.best_params_)
    # evaluate the model on test data
    print("Test Accuracy: ", random_search.score(X_test, y_test))
    return random_search

random_search(X_train, y_train, X_test, y_test, param_grid)

/var/folders/6s/hmlg0x1n07x_fl9blny_v37r0000gn/T/ipykernel_25108/202071377.py:15: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  random_search = RandomizedSearchCV(estimator=KerasClassifier(build_fn=create_google_model, verbose=0),


Fitting 3 folds for each of 10 candidates, totalling 30 fits



KeyboardInterrupt

